### Análisis EDA

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import sys
import io

sys.path.append('../scripts')
from dataset_loader import get_dataframe

warnings.filterwarnings('ignore')
plt.rcParams["font.family"] = "monospace"

In [ ]:
# Se carga el DataFrame principal, el cual contiene todos los datasets unificados
main_df = get_dataframe('../data/raw')


In [ ]:
# Funciones utilities para el análsis exploratorio de los datos.

def analisis_descriptivo(df):
	return pd.DataFrame([
			df.count(),
			df.nunique(),
			df.sum(numeric_only=True),
			df.mean(numeric_only=True),
			df.median(numeric_only=True),
			df.mode(numeric_only=True).iloc[0],
			df.min(numeric_only=True),
			df.max(numeric_only=True),
			df.std(numeric_only=True),
			df.sem(numeric_only=True),
			df.var(numeric_only=True),
			df.skew(numeric_only=True),
			df.kurt(numeric_only=True),
			df.select_dtypes(include=np.number).quantile(0.25),
			df.select_dtypes(include=np.number).quantile(0.75),
		],
		index=["count", "num unique", "sum", "mean", "median", "mode", "min", "max", "std dev.", "std error", "variance", "skewness", "kurtosis", "quantile 25%", "quantile 75%"]
	)

def matriz_correlacion(df):
	df = df.select_dtypes(include=np.number).copy()
	matrix = df.corr()
	
	plt.figure(figsize=(8,6))
	sns.heatmap(matrix, linewidth=0.5,cmap="RdYlGn", vmax=1, vmin=-1, annot=matrix.to_numpy(), annot_kws={"size":6})
	plt.title("Matriz de Correlación", fontsize=12)
	
	buf = io.StringIO()
	plt.savefig(buf, format="svg")
	
	return {
		"matrix": matrix,
		"svg": buf.getvalue(),
	}

def crear_boxplot(df):
	df = df.select_dtypes(include=np.number).copy()
	
	for column in df.columns:
		df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
	
	plt.figure(figsize=(8, 6))
	ax = sns.boxplot(x="variable", y="value", data=pd.melt(df))
	ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
	
	plt.title("Detectar outliers")
	plt.xlabel("Variables")
	plt.ylabel("Valores")
	plt.show()

def descriptivo_coleccion(df_source, categoria):
    stats_merged = pd.DataFrame()
    
    for i,n in enumerate(df_source[categoria].cat.categories, 1):
        df = df_source[df_source[categoria + '_id'] == i][['costo']]

        if df.size:
            df = analisis_descriptivo(df)
            df.columns = [n]
            stats_merged = pd.concat([stats_merged, df], axis=1)
    
    return stats_merged

In [ ]:
# Se muestran las primeras 10 filas del dataset.
display(main_df.head(10))

# Generación de descriptivos por cada supermercado.
display(descriptivo_coleccion(main_df, 'supermercado'))

# Generación de descriptivos por cada producto.
display(descriptivo_coleccion(main_df, 'producto'))

# Generación de una matriz de correlación de las variables numéricas.
# Aparte de la obvia relación entre cadena y supermercado, no se evidencia
# correlación significativa entre las variables.
result = matriz_correlacion(main_df)
display(result['svg'])

# Generación de un boxplot para visualizar outliers de las variables numéricas.
crear_boxplot(main_df)


In [ ]:
# Función utility para remover outliers. Recibe el dataframe, la variable x la cual se va a truncar,
# la variable y de filtrado, y el threshold permitido para la desviación estandar.
def remove_outliers(df_source, x_variable, y_variable, scale_std=1.0):
	ret = pd.DataFrame()

	for id,m in enumerate(df_source[y_variable].cat.categories):
		sub_df = df_source[df_source[y_variable + '_id'] == id]
		
		data_m, data_std = np.mean(sub_df[x_variable]), np.std(sub_df[x_variable])
		cutoff = data_std * scale_std
		v0, v1 = data_m - cutoff, data_m + cutoff
		sub_df = sub_df[(sub_df[x_variable] > v0) & (sub_df[x_variable] < v1)]
		ret = pd.concat([ret, sub_df])
	
	return ret

# Variables a considerar. Se debe de escoger costo como variable en x, ya que esta es la que vamos a truncar,
# pero se escoge producto para este análisis. El enfoque del estudio se hará principalmente sobre costos por producto de la
# CBA. Los supermercados puede ser una segunda opción pero para otro estudio, ya que abarcarían todos los costos
# de todos los productos de la CBA que ofrecen.
xvar = 'costo'
yvar = 'producto'

# Recortar outliers a 2 veces la desviación estandar.
main_df_no_outlier = remove_outliers(main_df, xvar, yvar, scale_std=2)

plt.figure(figsize=(12,15))
sns.boxplot(data=main_df, y=yvar, x=xvar)
plt.title('Boxplot original - {} vs {}'.format(yvar.capitalize(), xvar.capitalize()))
plt.show()

plt.figure(figsize=(12,15))
sns.boxplot(data=main_df_no_outlier, y=yvar, x=xvar)
plt.title('Boxplot filtrado - {} vs {}'.format(yvar.capitalize(), xvar.capitalize()))
plt.show()

In [ ]:
# Generación de gráficos para cada cadena de supermercados. Cada gráfico contiene sub-gráficos, uno
# por cada producto de la CBA ofertado por la cadena. Todas los supermercados tipo abarrotería, minisuper,
# kioskos, etc que no pertenezcan a alguna cadena reconocida, fueron agrupados bajo el nombre 'Otros'.
# Aviso. Este código toma cierto tiempo en culminar.

# Se recorren todas las cadenas de supermercado.
for cadena_id,cadena_name in enumerate(main_df.cadena.cat.categories):
    producto_dfs = []

    # Se recorren todos los productos de esta cadena.
    for producto_id,producto_name in enumerate(main_df.producto.cat.categories):
        df = main_df[(main_df.producto_id == producto_id) & (main_df.cadena_id == cadena_id)]

        # Se agrupan los productos por año y mes, para luego obtener la media de los costos.
        df = df.groupby(['anio', 'mes'])['costo'].mean().reset_index()
        df['fecha'] = pd.to_datetime(df['anio'].astype(str) + '-' + df['mes'].astype(str))
        producto_dfs.append((df, producto_name))
    
    # Se prepara un sublot de 3 columnas por n filas de productos.
    items_per_chunk = 3
    chunks = [producto_dfs[i:i+items_per_chunk] for i in range(0, len(producto_dfs), items_per_chunk)]
    
    fig, axs = plt.subplots(len(chunks), items_per_chunk, figsize=(15,40))
    fig.suptitle('Variación de costos de varios productos para cadena de supermercados %s' % cadena_name)

    for chu_idx, chu in enumerate(chunks):
        ax_chu = axs[chu_idx]

        for i,ax in enumerate(ax_chu):
            try: df, producto_name = chu[i]
            except:
                ax.set_axis_off()
                continue

            ax.plot(df['fecha'], df['costo'], marker='o')
            ax.set_title(producto_name, fontsize=6)
            ax.tick_params(axis='x', labelsize=6)
            ax.tick_params(axis='y', labelsize=6)
            ax.grid(True, color='#dedede')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.96)
    plt.show()
